# Workshop Part 1: The "Hands" (MCP Server)

In this notebook, we will build a **Code Utility Server**.
Instead of running this inside the notebook, we will use the `%%writefile` magic command to generate a production-ready `server.py` file.

**Goal:** Create a server with two tools:
1. `run_safe_python`: Executes Python code in a sandbox.
2. `lint_code`: Checks code for syntax errors (The Evaluator).

In [ ]:
# Install necessary libraries
%pip install fastmcp uvicorn

In [ ]:
%%writefile server.py
from fastmcp import FastMCP
import ast
import sys
import io

# 1. Initialize the Server
# We use 'fastmcp' which handles the JSON-RPC and SSE transport automatically.
mcp = FastMCP("CodeAssistant")

# 2. Define Tool: The "Evaluator" (Linter)
@mcp.tool()
def lint_code(code: str) -> str:
    """
    Static analysis tool. Checks Python code for syntax errors without executing it.
    Returns 'OK' or the error message.
    """
    try:
        ast.parse(code)
        return "OK"
    except SyntaxError as e:
        return f"Syntax Error on line {e.lineno}: {e.msg}"

# 3. Define Tool: The "Actuator" (Safe Execution)
@mcp.tool()
def run_safe_python(code: str) -> str:
    """
    Executes Python code and returns the stdout.
    WARNING: For workshop demo only. Use strict sandboxing in prod.
    """
    # Create a buffer to capture stdout
    old_stdout = sys.stdout
    redirected_output = sys.stdout = io.StringIO()

    try:
        # Define a restricted global scope
        safe_globals = {"print": print, "range": range, "len": len}
        exec(code, safe_globals)
    except Exception as e:
        return f"Runtime Error: {e}"
    finally:
        sys.stdout = old_stdout

    return redirected_output.getvalue()

# 4. Entry Point
if __name__ == "__main__":
    # fastmcp handles the 'run' command to serve via SSE or Stdio
    mcp.run()

### Running the Server
Now, open your terminal (or a separate tab) and run the server using HTTP/SSE transport:

```bash
fastmcp run server.py --transport sse --port 8000
```

You should see: `Server running at http://localhost:8000/sse`